# KM003C PD Protocol Analysis

This notebook focuses on analyzing the USB Power Delivery (PD) protocol functionality of the KM003C device.

**Step by step investigation of PD files:**
- `orig_with_pd.13`
- `pd_capture_new.9`

In [3]:
import polars as pl
import sys
from pathlib import Path

# Setup paths
project_root = Path.cwd()
while not (project_root / 'pyproject.toml').exists():
    project_root = project_root.parent
    
analysis_scripts_path = project_root / 'analysis' / 'scripts'
sys.path.insert(0, str(analysis_scripts_path))

import helpers

# Load dataset
df = helpers.load_master_dataset(project_root / 'usb_master_dataset.parquet')

print("Dataset loaded successfully!")
print(f"Total packets: {len(df)}")
print(f"Source files: {df['source_file'].unique().to_list()}")

✅ Loaded 12,008 USB packets from /home/okhsunrog/code/km003c-protocol-research/usb_master_dataset.parquet
Dataset loaded successfully!
Total packets: 12008
Source files: ['orig_adc_1000hz.6', 'orig_adc_50hz.6', 'orig_with_pd.13', 'orig_new_adcsimple0.12', 'pd_capture_new.9', 'orig_adc_record.6', 'orig_open_close.16', 'rust_simple_logger.16']


In [4]:
# Step 1: Examine the two PD files

pd_files = ['orig_with_pd.13', 'pd_capture_new.9']

for pd_file in pd_files:
    print(f"\n{'='*60}")
    print(f"FILE: {pd_file}")
    print('='*60)
    
    # Filter for this PD file
    pd_df = df.filter(pl.col('source_file') == pd_file)
    
    print(f"Packets in {pd_file}: {len(pd_df)}")
    print(f"Time range: {pd_df['timestamp'].min():.3f}s - {pd_df['timestamp'].max():.3f}s")
    print(f"Duration: {pd_df['timestamp'].max() - pd_df['timestamp'].min():.3f}s")
    
    # Show packet overview
    packet_overview = pd_df.group_by(['transfer_type', 'direction']).agg([
        pl.len().alias('count'),
        pl.col('data_length').mean().alias('avg_length')
    ]).sort('count', descending=True)
    
    print(f"\nPacket types in {pd_file}:")
    print(packet_overview)


FILE: orig_with_pd.13
Packets in orig_with_pd.13: 2056
Time range: 0.000s - 42.093s
Duration: 42.093s

Packet types in orig_with_pd.13:
shape: (3, 4)
┌───────────────┬───────────┬───────┬────────────┐
│ transfer_type ┆ direction ┆ count ┆ avg_length │
│ ---           ┆ ---       ┆ ---   ┆ ---        │
│ str           ┆ str       ┆ u32   ┆ f64        │
╞═══════════════╪═══════════╪═══════╪════════════╡
│ 0x03          ┆ D->H      ┆ 1022  ┆ 15.225049  │
│ 0x03          ┆ H->D      ┆ 1008  ┆ 2.15873    │
│ 0x02          ┆ D->H      ┆ 26    ┆ 13.038462  │
└───────────────┴───────────┴───────┴────────────┘

FILE: pd_capture_new.9
Packets in pd_capture_new.9: 6930
Time range: 0.000s - 295.600s
Duration: 295.600s

Packet types in pd_capture_new.9:
shape: (3, 4)
┌───────────────┬───────────┬───────┬────────────┐
│ transfer_type ┆ direction ┆ count ┆ avg_length │
│ ---           ┆ ---       ┆ ---   ┆ ---        │
│ str           ┆ str       ┆ u32   ┆ f64        │
╞═══════════════╪═══════════╪═

## PD Protocol Investigation Notes

Use this section to document findings about the KM003C PD protocol functionality:

### Key Observations
- [ ] PD packet identification patterns
- [ ] Command/response structures  
- [ ] Timing characteristics
- [ ] Error handling patterns

### Protocol Structure
- [ ] PD message types discovered
- [ ] Payload format analysis
- [ ] Transaction flow patterns

### Next Steps
- [ ] Implement PD-specific packet parsing
- [ ] Add PD message decoding
- [ ] Create PD visualization tools